# Process MEDImage obj. and save to npy
- 1_medimages_mets: contains medimage obj with combined mets masks
- 1_medimages_brain: contrains medimage oj

In [1]:
import sys
import os
import pydicom
from pathlib import Path
import scipy.io


import matplotlib.pyplot as plt
import time
import pandas as pd
import csv
import math
import numpy as np
import glob

import helper
import pickle


In [2]:
pythonCodePATH = '../../MEDomicsLab-develop/Code'
sys.path.append(pythonCodePATH)
from MEDimage.MEDimage import MEDimage
import ImageProcessing
from Utilities.ImageVolumeObj import ImageVolumeObj

Object not found in module


# Load medimage obj for mets and brain masks

In [53]:
# MEDImage obj with mets masks
dirLoc = '/data/public/MIM_BMETS_V6/1_medimage_mets'

miMets = {}
c=1500
fnames = glob.glob(dirLoc + '/' + '*.obj')
fnames.sort()
for name in fnames[c:]:
    FILE = open(name, 'rb')
    miMets[c] = pickle.load(FILE)
    FILE.close()
    c = c + 1;


In [56]:
# MEDImage obj with brain masks
dirLoc = '/data/public/MIM_BMETS_V6/1_medimage_brain'

miBrain = {}
c=1500
fnames = glob.glob(dirLoc + '/' + '*.obj')
fnames.sort()
for name in fnames[c:]:
    FILE = open(name, 'rb')
    miBrain[c] = pickle.load(FILE)
    FILE.close()
    c = c + 1;


In [32]:
def z_pad(image):
    depth, height, width = image.data.shape
    if depth == 64: return image
    if depth > 64:
        target = depth - 64
        if target % 2 == 0:
            n_pad_up, n_pad_down = int(target / 2), int(target / 2)
        else:
            n_pad_up, n_pad_down = int(target // 2) + 1, int(target // 2)
        image.data = image.data[n_pad_up:depth-n_pad_down]
    else:
        target = 64 - depth
        if target % 2 == 0:
            n_pad_up, n_pad_down = int(target / 2), int(target / 2)
        else:
            n_pad_up, n_pad_down = int(target // 2) + 1, int(target // 2)
        up_voxel = np.zeros((n_pad_up, height, width))
        down_voxel = np.zeros((n_pad_down, height, width))
        image.data = np.concatenate([up_voxel, image.data, down_voxel])
        
    return image

# Process images and masks

In [57]:
outF = open("slice_px_py.txt", "w")
c = 1500
for i in range(c,2138):
    print(fnames[i])
    
    # get images and masks
    mImage0 = {}
    mImageSS0 = {}
    mMetsMask0 = {}
    mBrainMask0 = {}
    mImage0, mMetsMask0 = (miMets[i]).get_image_and_mask()
    mImage0, mBrainMask0 = (miBrain[i]).get_image_and_mask()
    mImageSS0 = ImageVolumeObj(data=helper.extract_brain(mImage0.data, mBrainMask0.data), spatialRef=mImage0.spatialRef)
    print('0:', mImage0.data.shape, mMetsMask0.data.shape)
    
    line = str(miMets[i].wrapped_image[2][0].SliceThickness) + ',' + str(miMets[i].wrapped_image[2][0].PixelSpacing[0]) + ',' + str(miMets[i].wrapped_image[2][0].PixelSpacing[1])
    outF.write(line)
    outF.write("\n")
    
    # scale each image to have voxel size 1.5mm X 1.5mm X 3mm
    mImage1 = {}
    mImageSS1 = {}
    mMetsMask1 = {}
    mBrainMask1 = {}
    t = ImageVolumeObj(data=np.transpose(mImage0.data, (1,2,0)), spatialRef=mImage0.spatialRef)
    mImage1 = ImageProcessing.interpVolume.interpVolume(t,voxDim=[1.5,1.5,3],interpMet='cubic',image_type='image')
    mImage1.data = np.transpose(mImage1.data, (2,0,1))
    t = ImageVolumeObj(data=np.transpose(mImageSS0.data, (1,2,0)), spatialRef=mImage0.spatialRef)
    mImageSS1 = ImageProcessing.interpVolume.interpVolume(t,voxDim=[1.5,1.5,3],interpMet='cubic',image_type='image')
    mImageSS1.data = np.transpose(mImageSS1.data, (2,0,1))
    t = ImageVolumeObj(data=np.transpose(mMetsMask0.data, (1,2,0)), spatialRef=mImage0.spatialRef)
    mMetsMask1 = ImageProcessing.interpVolume.interpVolume(t,voxDim=[1.5,1.5,3],interpMet='linear',roundVal=0.5,image_type='roi')
    mMetsMask1.data = np.transpose(mMetsMask1.data, (2,0,1))
    t = ImageVolumeObj(data=np.transpose(mBrainMask0.data, (1,2,0)), spatialRef=mImage0.spatialRef)
    mBrainMask1 = ImageProcessing.interpVolume.interpVolume(t,voxDim=[1.5,1.5,3],interpMet='linear',roundVal=0.5,image_type='roi')
    mBrainMask1.data = np.transpose(mBrainMask1.data, (2,0,1))
    print('1:', mImage1.data.shape, mMetsMask1.data.shape)
    
    
    # size each image to 256 X 256 X 64
    mImage2 = {}
    mImageSS2 = {}
    mMetsMask2 = {}
    mBrainMask2 = {}
    mImage2 = helper.xy_pad(mImage1); mImage2 = z_pad(mImage2)
    mImageSS2 = helper.xy_pad(mImageSS1); mImageSS2 = z_pad(mImageSS2)
    mMetsMask2 = helper.xy_pad(mMetsMask1); mMetsMask2 = z_pad(mMetsMask2)
    mBrainMask2 = helper.xy_pad(mBrainMask1); mBrainMask2 = z_pad(mBrainMask2)
    print('2:', mImage2.data.shape, mMetsMask2.data.shape)
    
    # normalize images
    mImage3 = {}
    mImageSS3 = {}
    mMetsMask3 = {}
    mBrainMask3 = {}
    mImage3 = ImageVolumeObj(data=helper.normalize(mImage2.data, mBrainMask2.data),spatialRef=mImage0.spatialRef)
    mImageSS3 = ImageVolumeObj(data=helper.normalize(mImageSS2.data, mBrainMask2.data),spatialRef=mImage0.spatialRef) #helper.normalize(mImageSS2, mBrainMask2)
    mMetsMask3 = mMetsMask2
    mBrainMask3 = mBrainMask2
    print('3:', mImage3.data.shape, mMetsMask3.data.shape)
    
    # save to npy array
    t = fnames[i].split('/')
    t2 = t[-1].split('_')[0] +  '_' + t[-1].split('_')[1]
    np.save('/data/public/MIM_BMETS_V6/2_processed/brain_masks/' + t2, mBrainMask3.data)
    np.save('/data/public/MIM_BMETS_V6/2_processed/mets_masks/' + t2, mMetsMask3.data)
    np.save('/data/public/MIM_BMETS_V6/2_processed/images/' + t2, mImage3.data)
    np.save('/data/public/MIM_BMETS_V6/2_processed/skull_stripped/' + t2, mImageSS3.data)
    
outF.close()

/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01523_20130703_MEDimage_object.obj
0: (60, 512, 512) (60, 512, 512)
1: (60, 200, 200) (60, 200, 200)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01524_20131122_MEDimage_object.obj
0: (56, 704, 704) (56, 704, 704)
1: (56, 200, 200) (56, 200, 200)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01525_20130801_MEDimage_object.obj
0: (60, 512, 512) (60, 512, 512)
1: (60, 200, 200) (60, 200, 200)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01525_20160505_MEDimage_object.obj
0: (106, 256, 256) (106, 256, 256)
1: (53, 201, 201) (53, 201, 201)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01526_20130720_MEDimage_object.obj
0: (6

1: (50, 200, 200) (50, 200, 200)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01542_20140817_MEDimage_object.obj
0: (51, 704, 704) (51, 704, 704)
1: (51, 200, 200) (51, 200, 200)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01542_20141209_MEDimage_object.obj
0: (200, 512, 512) (200, 512, 512)
1: (100, 200, 200) (100, 200, 200)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01543_20140218_MEDimage_object.obj
0: (60, 512, 512) (60, 512, 512)
1: (60, 200, 200) (60, 200, 200)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01543_20150315_MEDimage_object.obj
0: (128, 256, 256) (128, 256, 256)
1: (64, 201, 201) (64, 201, 201)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 

0: (54, 704, 704) (54, 704, 704)
1: (54, 200, 200) (54, 200, 200)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01563_20140621_MEDimage_object.obj
0: (52, 512, 512) (52, 512, 512)
1: (52, 200, 200) (52, 200, 200)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01563_20150210_MEDimage_object.obj
0: (56, 704, 704) (56, 704, 704)
1: (56, 200, 200) (56, 200, 200)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01564_20131122_MEDimage_object.obj
0: (54, 704, 704) (54, 704, 704)
1: (54, 200, 200) (54, 200, 200)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01564_20161027_MEDimage_object.obj
0: (104, 256, 256) (104, 256, 256)
1: (52, 201, 201) (52, 201, 201)
2: (64, 256, 256) (64, 256, 256)

3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01586_20140316_MEDimage_object.obj
0: (56, 704, 704) (56, 704, 704)
1: (56, 200, 200) (56, 200, 200)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01587_20140323_MEDimage_object.obj
0: (56, 704, 704) (56, 704, 704)
1: (56, 200, 200) (56, 200, 200)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01588_20140215_MEDimage_object.obj
0: (58, 704, 704) (58, 704, 704)
1: (58, 200, 200) (58, 200, 200)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01589_20140210_MEDimage_object.obj
0: (66, 704, 704) (66, 704, 704)
1: (66, 200, 200) (66, 200, 200)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01590_201

1: (58, 200, 200) (58, 200, 200)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01616_20140519_MEDimage_object.obj
0: (142, 256, 256) (142, 256, 256)
1: (71, 201, 201) (71, 201, 201)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01616_20140803_MEDimage_object.obj
0: (128, 512, 512) (128, 512, 512)
1: (64, 200, 200) (64, 200, 200)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01617_20140622_MEDimage_object.obj
0: (50, 704, 704) (50, 704, 704)
1: (50, 200, 200) (50, 200, 200)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01618_20140622_MEDimage_object.obj
0: (56, 704, 704) (56, 704, 704)
1: (56, 200, 200) (56, 200, 200)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 25

...

1: (50, 200, 200) (50, 200, 200)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01841_20160611_MEDimage_object.obj
0: (54, 704, 704) (54, 704, 704)
1: (54, 200, 200) (54, 200, 200)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01841_20160625_MEDimage_object.obj
0: (90, 400, 400) (90, 400, 400)
1: (60, 200, 200) (60, 200, 200)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01841_20161022_MEDimage_object.obj
0: (54, 704, 704) (54, 704, 704)
1: (54, 200, 200) (54, 200, 200)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01841_20170715_MEDimage_object.obj
0: (52, 704, 704) (52, 704, 704)
1: (52, 200, 200) (52, 200, 200)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/

1: (52, 200, 200) (52, 200, 200)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01864_20170719_MEDimage_object.obj
0: (52, 704, 704) (52, 704, 704)
1: (52, 200, 200) (52, 200, 200)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01865_20161108_MEDimage_object.obj
0: (56, 704, 704) (56, 704, 704)
1: (56, 200, 200) (56, 200, 200)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01866_20160908_MEDimage_object.obj
0: (56, 704, 704) (56, 704, 704)
1: (56, 200, 200) (56, 200, 200)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01866_20160925_MEDimage_object.obj
0: (56, 704, 704) (56, 704, 704)
1: (56, 200, 200) (56, 200, 200)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/

3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01889_20161110_MEDimage_object.obj
0: (98, 256, 256) (98, 256, 256)
1: (49, 201, 201) (49, 201, 201)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01890_20161018_MEDimage_object.obj
0: (98, 256, 256) (98, 256, 256)
1: (49, 201, 201) (49, 201, 201)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01890_20171114_MEDimage_object.obj
0: (126, 256, 256) (126, 256, 256)
1: (63, 147, 147) (63, 147, 147)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01891_20161225_MEDimage_object.obj
0: (50, 512, 512) (50, 512, 512)
1: (50, 200, 200) (50, 200, 200)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01892_2

1: (64, 200, 200) (64, 200, 200)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01918_20170414_MEDimage_object.obj
0: (106, 256, 256) (106, 256, 256)
1: (53, 201, 201) (53, 201, 201)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01918_20171006_MEDimage_object.obj
0: (110, 256, 256) (110, 256, 256)
1: (55, 147, 147) (55, 147, 147)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01919_20170415_MEDimage_object.obj
0: (128, 256, 256) (128, 256, 256)
1: (64, 201, 201) (64, 201, 201)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01920_20170302_MEDimage_object.obj
0: (56, 704, 704) (56, 704, 704)
1: (56, 200, 200) (56, 200, 200)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 

3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01951_20170806_MEDimage_object.obj
0: (51, 704, 704) (51, 704, 704)
1: (51, 200, 200) (51, 200, 200)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01953_20171115_MEDimage_object.obj
0: (128, 512, 512) (128, 512, 512)
1: (86, 147, 147) (86, 147, 147)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01954_20171012_MEDimage_object.obj
0: (126, 256, 256) (126, 256, 256)
1: (63, 147, 147) (63, 147, 147)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01955_20170930_MEDimage_object.obj
0: (126, 256, 256) (126, 256, 256)
1: (63, 147, 147) (63, 147, 147)
2: (64, 256, 256) (64, 256, 256)
3: (64, 256, 256) (64, 256, 256)
/data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-019

In [51]:
#804, /data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01056_20080109_MEDimage_object.obj, has no mets roi
#1054, /data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01212_20130430_MEDimage_object.obj, none obj
#1060, /data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01212_20140512_MEDimage_object.obj, none
#1064, /data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01212_20150418_MEDimage_object.obj, none
#1066, /data/public/MIM_BMETS_V6/1_medimage_brain/BrainMets-UCSF-01212_20150719_MEDimage_object.obj
i

1066

In [70]:
import plotly.graph_objects as go

In [54]:
import plotly.io as pio

In [8]:
import plotly.express as px
from plotly.subplots import make_subplots


In [9]:
from ipywidgets import FloatSlider, interactive

In [12]:
w = FloatSlider(
    value=32,
    min=0,#mImageSS3.data[0],
    max=63,#mImageSS3.data[-1],
    step=1,#np.diff(mImageSS3.data)[0],
    description='Slice Position (mm):',
)

def InteractiveReview(i):
    fig = px.imshow(mImageSS3.data[int(i)], color_continuous_scale='gray')
    fig.update_layout(height=350, width=600)
    fig.show()

In [13]:
interactive(InteractiveReview, i=w)

interactive(children=(FloatSlider(value=32.0, description='Slice Position (mm):', max=63.0, step=1.0), Output(…